### --- 0. Πειραματικό Πλαίσιο και Οδηγίες Εκτέλεσης ---

Το παρόν script αποτελεί το θεμελιώδες βήμα για τη συστηματική εξαγωγή ενός ευρέος φάσματος ακουστικών χαρακτηριστικών από αρχεία ήχου. Είναι σχεδιασμένο για να λειτουργεί ως μέρος μιας ευρύτερης ερευνητικής διαδικασίας που περιλαμβάνει την ανάλυση δεδομένων από 154 ασθενείς, καθένας από τους οποίους αξιολογήθηκε σε δύο διακριτές φαρμακευτικές καταστάσεις: **'ON'** (υπό αγωγή) και **'OFF'** (χωρίς αγωγή).

#### **Πρωτόκολλο Εκτέλεσης:**
Για να διασφαλιστεί ο απόλυτος διαχωρισμός μεταξύ των δεδομένων των δύο καταστάσεων, το script αυτό εκτελέστηκε επαναληπτικά και ανεξάρτητα για κάθε συνθήκη. Συγκεκριμένα:

1.  **Εκτελέστηκε 154 φορές:** μία φορά για κάθε ασθενή, χρησιμοποιώντας το σύνολο των αρχείων ήχου που ηχογραφήθηκαν στην κατάσταση **'ON'**.

2.  **Εκτελέστηκε επιπλέον 154 φορές:** μία φορά για κάθε ασθενή, χρησιμοποιώντας το σύνολο των αρχείων ήχου που ηχογραφήθηκαν στην κατάσταση **'OFF'**.

Αυτή η μεθοδική προσέγγιση εγγυάται ότι τα χαρακτηριστικά για τις καταστάσεις 'ON' και 'OFF' επεξεργάζονται και αποθηκεύονται σε εντελώς ξεχωριστά αρχεία, αποτρέποντας την ανάμιξη δεδομένων και διευκολύνοντας την επακόλουθη στατιστική σύγκριση και μοντελοποίηση.

In [1]:
# --- 1. Εισαγωγή Βιβλιοθηκών ---
import os
import librosa
import pandas as pd
import numpy as np
import parselmouth
from parselmouth.praat import call
from scipy.stats import entropy
import opensmile

# --- 2. Ορισμός Αρχείων και Παραμέτρων ---
# Λίστα με τα αρχεία ήχου προς επεξεργασία.
audio_files = [
    "a1.wav", "a2.wav", "a3.wav", "aui.wav", "ball1.wav", "ball2.wav", "breathing.wav",
    "coughing.wav", "counting.wav", "earth.wav", "f_autoind.wav", "f_bat.wav", "f_bicycle.wav",
    "f_cinema.wav", "f_cockroachkiller.wav", "f_mailman.wav", "f_pastrychef.wav", "f_radio.wav",
    "f_ring.wav", "f_rose.wav", "f_safe.wav", "f_snail.wav", "f_spaceship.wav", "f_telecom.wav",
    "illness.wav", "ka.wav", "kala.wav", "lentils.wav", "mom1.wav", "mom2.wav", "oe.wav", "pa.wav",
    "pataka.wav", "rainbow_passage.wav", "s_autoind.wav", "s_bat.wav", "s_bicycle.wav", "s_cinema.wav",
    "s_cockroachkiller.wav", "s_mailman.wav", "s_pastrychef.wav", "s_radio.wav", "s_ring.wav", "s_rose.wav", "s_safe.wav", "s_snail.wav", "s_spaceship.wav",
    "s_telecom.wav", "s1.wav", "s2.wav", "s3.wav", "sakis.wav", "say.wav", "school.wav", "ta.wav",
    "wind.wav", "yesno.wav", "z1.wav", "z2.wav", "z3.wav"
]

# Όνομα του αρχείου CSV εξόδου όπου θα αποθηκευτούν τα εξαγόμενα χαρακτηριστικά.
OUTPUT_FILE = "extracted_features_1.csv"

# --- 3. Ορισμός Συναρτήσεων Εξαγωγής Χαρακτηριστικών ---
# Συνάρτηση για χαρακτηριστικά δυναμικών μεταβολών της ενέργειας.
def extract_dynamic_energy(y, sr):
     # Υπολογισμός της έντασης (RMS - Root Mean Square).
    intensity = librosa.feature.rms(y=y)
     # Τυπική απόκλιση της έντασης.
    intensity_std = np.std(intensity)
    # Εύρος της έντασης (μέγιστη - ελάχιστη τιμή).
    intensity_range = np.ptp(intensity)
    # Soft Phonation Index (SPI): Μέσος όρος των τιμών έντασης που είναι κάτω από τη διάμεσο.
    spi = np.mean(intensity[intensity < np.median(intensity)])
    # Voice Tremor Index (VTI): Μέσος όρος των τιμών έντασης που είναι πάνω από τη διάμεσο.
    vti = np.mean(intensity[intensity > np.median(intensity)])
    return intensity_std, intensity_range, spi, vti

# Υπολογισμός του Τελεστή Ενέργειας Teager (Teager Energy Operator).
def teager_energy_operator(signal):
    return signal[1:-1] ** 2 - signal[2:] * signal[:-2]

# Συνάρτηση για χαρακτηριστικά βασισμένα στον Τελεστή Ενέργειας Teager.
def extract_teager_energy(y):
    teager_energy = teager_energy_operator(y)
    mean_teager = np.mean(teager_energy)
    var_teager = np.var(teager_energy)
    teager_entropy = entropy(teager_energy) if np.sum(teager_energy) > 0 else 0
    return mean_teager, var_teager, teager_entropy

# Συνάρτηση για χαρακτηριστικά φωνητικού τρόμου (vocal tremor)
def extract_vocal_tremor(y, sr):
    tremor_frequency = librosa.feature.zero_crossing_rate(y)
    tremor_amplitude_variability = np.std(tremor_frequency)
    tremor_intensity_index = np.mean(tremor_frequency)
    return tremor_frequency.mean(), tremor_amplitude_variability, tremor_intensity_index

# Συνάρτηση για HNR (Harmonics-to-Noise Ratio) και σχετικά μέτρα, χρησιμοποιώντας το Parselmouth (Praat).
def extract_hnr(sound):
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1)
    hnr = call(harmonicity, "Get mean", 0, 0)
    nhr = 1 / (hnr + 1e-10)
    spi = hnr / 10  # Simplified calculation
    return hnr, nhr, spi

# Συνάρτηση για χαρακτηριστικά τονικού ύψους (pitch), χρησιμοποιώντας το Parselmouth (Praat).
def extract_pitch(sound):
    pitch = call(sound, "To Pitch", 0.0, 75, 600)
    mean_pitch = call(pitch, "Get mean", 0, 0, "Hertz")
    pitch_min = call(pitch, "Get minimum", 0, 0, "Hertz", "Parabolic")
    pitch_max = call(pitch, "Get maximum", 0, 0, "Hertz", "Parabolic")
    pitch_range = pitch_max - pitch_min
    pitch_strength = np.mean(np.abs(pitch.selected_array['frequency'] - mean_pitch))
    pitch_entropy = entropy(pitch.selected_array['frequency']) if np.sum(pitch.selected_array['frequency']) > 0 else 0
    return mean_pitch, pitch_range, pitch_strength, pitch_entropy

# Συνάρτηση για χαρακτηριστικά προσωδίας (speech prosody).
def extract_speech_prosody(y, sr):
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
    speech_rate = tempo / 60.0
    rhythm_var = np.std(beats)
    return speech_rate, rhythm_var

# Συνάρτηση για χαρακτηριστικά μορφοφώνων (formants), χρησιμοποιώντας το Parselmouth (Praat).
def extract_formants(sound):
    formant = call(sound, "To Formant (burg)", 0.0, 5, 5500, 0.025, 50)
    formant_1 = [call(formant, "Get value at time", 1, t, "Hertz", "Linear") for t in formant.ts()]
    formant_2 = [call(formant, "Get value at time", 2, t, "Hertz", "Linear") for t in formant.ts()]
    formant_3 = [call(formant, "Get value at time", 3, t, "Hertz", "Linear") for t in formant.ts()]
    f1_mean, f1_std = np.mean(formant_1), np.std(formant_1)
    f2_mean, f2_std = np.mean(formant_2), np.std(formant_2)
    f3_mean, f3_std = np.mean(formant_3), np.std(formant_3)
    return f1_mean, f1_std, f2_mean, f2_std, f3_mean, f3_std

# Συνάρτηση για τον υπολογισμό του χρόνου φώνησης.
def extract_phonation_time(y):
    phonation_time = np.sum(librosa.effects.split(y, top_db=20)) / len(y)
    return phonation_time

# Συνάρτηση για τον υπολογισμό της διάρκειας και του αριθμού των παύσεων
def extract_pauses(y, sr):
    non_silent_intervals = librosa.effects.split(y, top_db=20)
    pause_durations = [(non_silent_intervals[i][0] - non_silent_intervals[i-1][1]) / sr for i in range(1, len(non_silent_intervals))]
    avg_pause_duration = np.mean(pause_durations) if pause_durations else 0
    num_pauses = len(pause_durations)
    return avg_pause_duration, num_pauses

# Συνάρτηση για τη δυναμική της ομιλίας
def extract_speech_dynamics(y):
    derivative = np.abs(np.diff(y))
    mean_derivative = np.mean(derivative)
    std_derivative = np.std(derivative)
    temporal_stability = std_derivative / mean_derivative
    return mean_derivative, std_derivative, temporal_stability

# Συνάρτηση για την εξαγωγή χαρακτηριστικών eGeMAPS με το openSMILE
def extract_egemaps(y, sr):
    # Αρχικοποίηση του openSMILE για το σετ χαρακτηριστικών eGeMAPS (ένα τυποποιημένο σετ για ανάλυση συναισθήματος και παθολογιών)
    smile = opensmile.Smile(
        feature_set=opensmile.FeatureSet.eGeMAPSv02,
        feature_level=opensmile.FeatureLevel.Functionals, # Υπολογίζει στατιστικά (π.χ. mean, std) για όλο το αρχείο
    )
     # Επεξεργασία του ηχητικού σήματος.
    features = smile.process_signal(y, sr)
    return features


# --- 4. Κύρια Διαδικασία Εξαγωγής ---
# Αρχικοποίηση ενός κενού DataFrame για την αποθήκευση όλων των χαρακτηριστικών.
# Ορίζουμε τις στήλες για τα χαρακτηριστικά που υπολογίσαμε χειροκίνητα.
features_df = pd.DataFrame(columns=[
    "file_name",
    "intensity_std", "intensity_range", "spi", "vti",
    "mean_teager", "var_teager", "teager_entropy",
    "tremor_frequency", "tremor_amplitude_variability", "tremor_intensity_index",
    "hnr", "nhr", "hnr_spi",
    "mean_pitch", "pitch_range", "pitch_strength", "pitch_entropy",
    "speech_rate", "rhythm_var",
    "f1_mean", "f1_std", "f2_mean", "f2_std", "f3_mean", "f3_std",
    "phonation_time",
    "avg_pause_duration", "num_pauses",
    "mean_derivative", "std_derivative", "temporal_stability",
    "empty_column"
])

# Κύριος βρόχος που επαναλαμβάνεται για κάθε αρχείο ήχου στη λίστα.
for audio_file in audio_files:
    try:
        #load the audio files
        y, sr = librosa.load(audio_file, sr=None)
        sound = parselmouth.Sound(audio_file)
        
        # Κλήση όλων των συναρτήσεων εξαγωγής χαρακτηριστικών.
        intensity_std, intensity_range, spi, vti = extract_dynamic_energy(y, sr)
        mean_teager, var_teager, teager_entropy = extract_teager_energy(y)
        tremor_frequency, tremor_amplitude_variability, tremor_intensity_index = extract_vocal_tremor(y, sr)
        hnr, nhr, hnr_spi = extract_hnr(sound)
        mean_pitch, pitch_range, pitch_strength, pitch_entropy = extract_pitch(sound)
        speech_rate, rhythm_var = extract_speech_prosody(y, sr)
        f1_mean, f1_std, f2_mean, f2_std, f3_mean, f3_std = extract_formants(sound)
        phonation_time = extract_phonation_time(y)
        avg_pause_duration, num_pauses = extract_pauses(y, sr)
        mean_derivative, std_derivative, temporal_stability = extract_speech_dynamics(y)
        egemaps_features = extract_egemaps(y, sr)
        
        # Επεξεργασία των χαρακτηριστικών eGeMAPS για να προστεθούν στο DataFrame.
        # Μετατροπή των τιμών και των ονομάτων των χαρακτηριστικών eGeMAPS σε λίστες.
        egemaps_feature_values = egemaps_features.values.flatten().tolist()
        egemaps_feature_names = egemaps_features.columns.tolist()
        
        # Δημιουργία μιας νέας γραμμής (ως DataFrame) για να προστεθεί στα αποτελέσματα.
        feature_row = pd.DataFrame([{
             # Προσθήκη των "χειροκίνητων" χαρακτηριστικών.
            "file_name": audio_file,
            "intensity_std": intensity_std,
            "intensity_range": intensity_range,
            "spi": spi,
            "vti": vti,
            "mean_teager": mean_teager,
            "var_teager": var_teager,
            "teager_entropy": teager_entropy,
            "tremor_frequency": tremor_frequency,
            "tremor_amplitude_variability": tremor_amplitude_variability,
            "tremor_intensity_index": tremor_intensity_index,
            "hnr": hnr,
            "nhr": nhr,
            "hnr_spi": hnr_spi,
            "mean_pitch": mean_pitch,
            "pitch_range": pitch_range,
            "pitch_strength": pitch_strength,
            "pitch_entropy": pitch_entropy,
            "speech_rate": speech_rate,
            "rhythm_var": rhythm_var,
            "f1_mean": f1_mean,
            "f1_std": f1_std,
            "f2_mean": f2_mean,
            "f2_std": f2_std,
            "f3_mean": f3_mean,
            "f3_std": f3_std,
            "phonation_time": phonation_time,
            "avg_pause_duration": avg_pause_duration,
            "num_pauses": num_pauses,
            "mean_derivative": mean_derivative,
            "std_derivative": std_derivative,
            "temporal_stability": temporal_stability,
            "empty_column": None, #empty column between my features and egemaps features
            **dict(zip(egemaps_feature_names, egemaps_feature_values))
        }])
        # Συνένωση της νέας γραμμής με το κύριο DataFrame.
        features_df = pd.concat([features_df, feature_row], ignore_index=True)
    except Exception as e:
        # Χειρισμός σφαλμάτων: αν η επεξεργασία ενός αρχείου αποτύχει, εκτυπώνει το σφάλμα και συνεχίζει στο επόμενο.
        print(f"Error processing {audio_file}: {e}")

# --- 5. Αποθήκευση Αποτελεσμάτων ---
#save features to a CSV file
features_df.to_csv(OUTPUT_FILE, index=False)

# Εκτύπωση μηνύματος επιβεβαίωσης όταν η διαδικασία ολοκληρωθεί.
print("Feature extraction complete!", OUTPUT_FILE) #just to be sure that the extraction of the features is right


Feature extraction complete! extracted_features_1.csv
